In [1]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from dotenv import load_dotenv
load_dotenv()

True

In [14]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

# define LLM
llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
Settings.llm = llm
Settings.chunk_size = 512

In [16]:
from llama_index.core import KnowledgeGraphIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.graph_stores.neo4j import Neo4jGraphStore
from IPython.display import Markdown, display

documents = SimpleDirectoryReader(input_files=["data/paul_graham_essay.txt"]).load_data()

In [19]:
print(len(documents))
print(documents[0])

1
Doc ID: 42240b21-3e53-4f8c-adf9-c2099ddcdf1d
Text: What I Worked On  February 2021  Before college the two main
things I worked on, outside of school, were writing and programming. I
didn't write essays. I wrote what beginning writers were supposed to
write then, and probably still are: short stories. My stories were
awful. They had hardly any plot, just characters with strong feelings,
which I ...


In [13]:
from llama_index.core import KnowledgeGraphIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.graph_stores.neo4j import Neo4jGraphStore

graph_store = Neo4jGraphStore(
    username=username,
    password=password,
    url=url,
    database=database,
)

storage_context = StorageContext.from_defaults(graph_store=graph_store)

In [20]:
# NOTE: can take a while!
index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=2,
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/comp

In [21]:

query_engine = index.as_query_engine(
    include_text=False, response_mode="tree_summarize"
)

response = query_engine.query("Tell me more about Interleaf")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Index was not constructed with embeddings, skipping embedding usage...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [33]:
print(response)

Interleaf was a software company that developed desktop publishing and content management software.


In [31]:
# display(Markdown(f"{response}"))

# print(response)

for node in response.source_nodes:
    # print(node.node_id)
    print(f"Text: {node.text}\n")
    print(node.metadata)

Text: The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
['GOT_CRUSHED_BY', "Moore's law"]
['TAUGHT', 'What not to do']
['ADDED', 'Scripting language']
['MADE', 'Software']
['MADE', 'Software', 'WAS', "Raison d'etre"]

{'kg_rel_texts': ['[\'GOT_CRUSHED_BY\', "Moore\'s law"]', "['TAUGHT', 'What not to do']", "['ADDED', 'Scripting language']", "['MADE', 'Software']", '[\'MADE\', \'Software\', \'WAS\', "Raison d\'etre"]'], 'kg_rel_map': {'Interleaf': [['GOT_CRUSHED_BY', "Moore's law"], ['TAUGHT', 'What not to do'], ['ADDED', 'Scripting language'], ['MADE', 'Software'], ['MADE', 'Software', 'WAS', "Raison d'etre"]]}, 'kg_schema': {'schema': 'Node properties are the following:\n\nRelationship properties are the following:\n\nThe relationships are the following:\n'}}


In [37]:
query = "What did the author write their dissertation on?"
query = "Why did the author move to England?"
query = "Tell me more about England"

response = query_engine.query(query)
print(f"{response}\n\n")

for node in response.source_nodes:
    # print(node.node_id)
    print(f"Text: {node.text}")
    print(f"Meta: {node.metadata}")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Index was not constructed with embeddings, skipping embedding usage...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> No relationships found, returning nodes found by keywords.
> No relationships found, returning nodes found by keywords.
INFO:llama_index.core.indices.knowledge_graph.retrievers:> No nodes found by keywords, returning empty response.
> No nodes found by keywords, returning empty response.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
England is a country that is part of the United Kingdom. It is located in the southern part of the island of Great Britain and shares borders with Scotland to the north and Wales to the west. England has a rich history, with landmarks such

In [40]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever

graph_rag_retriever = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    verbose=True,
    with_nl2graphquery=True,
)

query_engine = RetrieverQueryEngine.from_args(
    graph_rag_retriever,
)

In [42]:
query = "What did the author write their dissertation on?"
query = "Why did the author move to England?"
query = "Tell me more about England"
query = "Tell me more about Interleaf"

response = query_engine.query(query)
print(f"{response}\n\n")

for node in response.source_nodes:
    # print(node.node_id)
    print(f"Text: {node.text}")
    print(f"Meta: {node.metadata}")

Error in retrieving from nl2graphquery: 'NoneType' object has no attribute 'kwargs'
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Interleaf was a software company that developed a desktop publishing program called Interleaf.


Text: The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...` extracted based on key entities as subject:
['GOT_CRUSHED_BY', "Moore's law"]
['TAUGHT', 'What not to do']
['ADDED', 'Scripting langu

In [43]:
import pprint

pp = pprint.PrettyPrinter()
pp.pprint(response.metadata)

{'3337b339-ba08-4795-9dc2-b3845b33ccbc': {'kg_rel_map': {'Interleaf': [['GOT_CRUSHED_BY',
                                                                        "Moore's "
                                                                        'law'],
                                                                       ['TAUGHT',
                                                                        'What '
                                                                        'not '
                                                                        'to '
                                                                        'do'],
                                                                       ['ADDED',
                                                                        'Scripting '
                                                                        'language'],
                                                                       ['MADE',
                  